# Organize directories

The first step in training a ML model on sequences of images acquired from the ProtectCode app is to ensure the directory structure is set up for subsequent data loading. 

These commands assume the following: 

1) That the top directory for the training data and this notebook are both located in ~/src/YPB-AI/ on the EC2 instance. To get a zipped directory from S3 to that directory on your EC2 instance, you can run the following bash command: "aws s3 cp s3://your-bucket-name/yourZipFile.zip ~/src/YPB-AI/yourZipFile.zip". And to unzip it, you can run "unzip yourZipFile.zip"

2) That the jupyter notebook server was opened from the ~/src/ directory on the EC2 instance via the command "jupyter notebook --ip=0.0.0.0 --no-browser".

3) That underneath the top directory (which I usually call "Training" or something of the like), there are a series of subdirectories named and organized by classification label. For instance, each subdirectory that contains true negative image sequences, according to my implementation, contains the word "Negative". Each subdirectory that contains saturated image sequences (which are also negative, but classified differently), contains the phrase "Sat". And subdirectories with true positives do not contain the word "Negative" or the phrase "Sat". 



In [ ]:
#TODO -- correct issue which causes top folders (XS_Sat2, for instance) to be converted into pickle files. XS_Sat2.pickle ends up being empty.
# As do the other subdirectories that fall under the "Saturated" classification.

In [1]:
import numpy as np
from glob import glob
import os, sys
import re

Set the name of your top data directory below. 

In [2]:
topDirName = 'Training'

Lump together all subdirectories with the same classification label. 

In [3]:
#Assumes current working directory is ~/src/YPB-AI
allSamples = np.array(glob(topDirName + '/*'))
os.system('mkdir ' + topDirName + '/Absent')
os.system('mkdir ' + topDirName + '/Present')
os.system('mkdir ' + topDirName + '/Saturated')

for sample in allSamples: 
    if 'Negative' in sample:
        cmdString = "mv " + sample + " " + topDirName + "/Absent"
    elif 'Sat' in sample:
        cmdString = "mv " + sample + " " + topDirName + "/Saturated"
    else:
        cmdString = "mv " + sample + " " + topDirName + "/Present"
    os.system(cmdString)

In [4]:
# Find 'vestigial' directories to be deleted after unpacking.
presentVestDirs = np.array(glob(topDirName + '/Present/*'))
absentVestDirs = np.array(glob(topDirName + '/Absent/*'))
satVestDirs = np.array(glob(topDirName + '/Saturated/*'))

# Unpack the JPG photo files into identifiable directories.
allPresentSamples = np.array(glob(topDirName + '/Present/*/*'))
for sample in allPresentSamples:
    # Grab the unique identifiers -- phone model, sample # and date/time
    digestedSample = re.split('/',sample)
    digestedTimestamp = re.split('_',digestedSample[3])
    newDir = digestedSample[2] + '_' + digestedTimestamp[1] + '_' + digestedTimestamp[2]
    # Make the new directory
    cmdString = 'mkdir ' + topDirName + '/Present/' + newDir
    os.system(cmdString)
    # Add the photos directly to it
    cmdString = 'mv ' + sample + '/Photos/*' + ' ' + topDirName + '/Present/' + newDir
    os.system(cmdString)
    
allAbsentSamples = np.array(glob(topDirName + '/Absent/*/*'))
for sample in allAbsentSamples:
    # Grab the unique identifiers -- phone model, sample # and date/time
    digestedSample = re.split('/',sample)
    digestedTimestamp = re.split('_',digestedSample[3])
    newDir = digestedSample[2] + '_' + digestedTimestamp[1] + '_' + digestedTimestamp[2]
    # Make the new directory
    cmdString = 'mkdir ' + topDirName + '/Absent/' + newDir
    os.system(cmdString)
    # Add the photos directly to it
    cmdString = 'mv ' + sample + '/Photos/*' + ' ' + topDirName + '/Absent/' + newDir
    os.system(cmdString)
    
allSatSamples = np.array(glob(topDirName + '/Saturated/*/*'))
for sample in allSatSamples:
    # Grab the unique identifiers -- phone model, sample # and date/time
    digestedSample = re.split('/',sample)
    digestedTimestamp = re.split('_',digestedSample[3])
    newDir = digestedSample[2] + '_' + digestedTimestamp[1] + '_' + digestedTimestamp[2]
    # Make the new directory
    cmdString = 'mkdir ' + topDirName + '/Saturated/' + newDir
    os.system(cmdString)
    # Add the photos directly to it
    cmdString = 'mv ' + sample + '/Photos/*' + ' ' + topDirName + '/Saturated/' + newDir
    os.system(cmdString)
    
    
# Delete vestigial directories.
for direc in presentVestDirs:
    os.system('rm -rf ' + direc)
    
for direc in absentVestDirs:
    os.system('rm -rf ' + direc)

for direc in satVestDirs:
    os.system('rm -rf' + direc)

Look through all directories and find image sequences containing the file "IMG_9999.JPG". If found, sort and rename to an appropriate image sequence. This is necessary because when iOS reaches "IMG_9999.JPG" it wraps back around to "IMG_0000.JPG" for the next in the sequence. If this is left as is, the sorting operation that follows will sort the images into the incorrect timing order. 

In [5]:
# Get the filenames.
imgSeqsToCorrect = os.popen('cd ~/src/YPB-AI/; find -name "*IMG_9999.JPG"').read()
# Split into unique entities.
imgSeqsToCorrect = re.split('\n',imgSeqsToCorrect)
# Drop the last entry, it's empty.
imgSeqsToCorrect.pop();

# Change any files that don't start with 'IMG_9' (i.e., any files that come after the wraparound) to start with 'J'.
# This will mean that when they are sorted, they will come correctly after the wraparound.
for sample in imgSeqsToCorrect:
    digestedSample = re.split('/',sample)
    pathToDir = os.path.join(digestedSample[0],digestedSample[1],digestedSample[2],digestedSample[3])
    filesInDir = os.listdir(pathToDir)
    for i in range(len(filesInDir)):
        if 'IMG_9' not in filesInDir[i]:
            newFile = filesInDir[i].replace('I','J',1)
            cmdString = 'mv ' + os.path.join(pathToDir,filesInDir[i]) + ' ' + os.path.join(pathToDir,newFile)
            os.system(cmdString)

# Use a CNN to transform image sequences to a pickle file and organize the results.

In [6]:
import torch
import torchvision.models as models
from PIL import Image
import pickle

Use a pretrained CNN model (here using DenseNet121 because it has worked well in the past) with the final classifier layer chopped off.

In [7]:
# Import the CNN model and move it to CUDA if available.
model = models.densenet121(pretrained=True)

for param in model.parameters():
    param.requires_grad = False
del model.classifier

if torch.cuda.is_available():
    model.to('cuda');

Define the function that will transform model output into a pickle file.

In [8]:
def transform_to_pickle(model,topdir):
    model.eval()
    ndx = 0
    topdirs = [direc for direc in os.listdir(topdir) if direc[0] is not '.']
    
    # Calculate the total number of stacks to transform.
    numStacks = 0
    for classDir in topdirs:
        thisClassDir = os.path.join(topdir,classDir)
        directories = [direc for direc in os.listdir(thisClassDir) if direc[0] is not '.']
        numStacks += len(directories)
    
    for classDir in topdirs:
        thisClassDir = os.path.join(topdir,classDir)
        directories = [direc for direc in os.listdir(thisClassDir) if direc[0] is not '.']
        # Do the transformation.
        for photoDir in directories:
            thisPhotoDir = os.path.join(thisClassDir, photoDir)
            files = [photofile for photofile in os.listdir(thisPhotoDir) if photofile.lower().endswith('jpg')]
            print("Transforming image stack {}".format(thisPhotoDir))
            files.sort()
            output = []
            # Make sure to appropriately transform the image size and normalize the RGB values, as the model expects. 
            for file in files:
                thisFile = os.path.join(thisPhotoDir,file)
                imgArr = np.float32(np.array(Image.open(thisFile).resize((224,224))).reshape([1,3,224,224]))
                imgArr[0][0] = ((imgArr[0][0] - 123.675)/58.395)
                imgArr[0][1] = ((imgArr[0][1] - 116.28)/57.12)
                imgArr[0][2] = ((imgArr[0][2] - 103.53)/57.375)
                if torch.cuda.is_available():
                    output.append(model(torch.cuda.FloatTensor(imgArr)))
                else:
                    output.append(model(torch.FloatTensor(imgArr)))

            FinalOutput = torch.FloatTensor(len(files),1,1024)
            for i in range(len(files)):
                FinalOutput[i] = output[i]
            pathStr = thisPhotoDir + '.pickle'
            with open(pathStr,'wb') as f:
                pickle.dump(FinalOutput,f)
                ndx += 1
                print("{}% complete.".format(ndx/numStacks*100))
    

Do the transformation.

In [9]:
transform_to_pickle(model,topDirName)

Transforming image stack Training/Absent/6S_Negative_Sample3_Light_16-Apr-2019_15-59-47
0.009386146048432513% complete.
Transforming image stack Training/Absent/6_Negative_Sample3_Back_Light_16-Apr-2019_13-17-37
0.018772292096865026% complete.
Transforming image stack Training/Absent/SE_Negative_Dark_15-Mar-2019_15-37-50
0.028158438145297542% complete.
Transforming image stack Training/Absent/6_Negative_Sample3_Back_Light_16-Apr-2019_14-11-36
0.03754458419373005% complete.
Transforming image stack Training/Absent/6_Negative_Dark_13-Mar-2019_13-01-20
0.04693073024216257% complete.
Transforming image stack Training/Absent/7_Negative_Dark_13-Mar-2019_16-21-25
0.056316876290595085% complete.
Transforming image stack Training/Absent/7_Negative_Dark_13-Mar-2019_17-56-35
0.0657030223390276% complete.
Transforming image stack Training/Absent/7_Negative_Dark_13-Mar-2019_16-47-38
0.0750891683874601% complete.
Transforming image stack Training/Absent/6_Negative_Sample3_Back_Light_16-Apr-2019_12-5

0.694574807584006% complete.
Transforming image stack Training/Absent/7_Negative_Dark_13-Mar-2019_18-11-57
0.7039609536324386% complete.
Transforming image stack Training/Absent/SE_Negative_Dark_15-Mar-2019_15-50-04
0.7133470996808711% complete.
Transforming image stack Training/Absent/XS_Negative_Sample3_Light_17-Apr-2019_12-18-07
0.7227332457293035% complete.
Transforming image stack Training/Absent/7_Negative_Random_30-May-2019_17-25-35
0.732119391777736% complete.
Transforming image stack Training/Absent/SE_Negative_Dark_15-Mar-2019_16-49-22
0.7415055378261686% complete.
Transforming image stack Training/Absent/7_Negative_Dark_13-Mar-2019_17-19-54
0.7508916838746011% complete.
Transforming image stack Training/Absent/SE_Negative_Dark_15-Mar-2019_16-45-22
0.7602778299230336% complete.
Transforming image stack Training/Absent/6_Negative_Dark_13-Mar-2019_10-30-46
0.7696639759714661% complete.
Transforming image stack Training/Absent/SE_Negative_Dark_15-Mar-2019_16-19-12
0.779050122019

1.389149615168012% complete.
Transforming image stack Training/Absent/6S_Negative_Dark_14-Mar-2019_09-30-18
1.3985357612164444% complete.
Transforming image stack Training/Absent/6_Negative_Dark_13-Mar-2019_13-36-46
1.4079219072648772% complete.
Transforming image stack Training/Absent/XS_Negative_Sample3_Light_17-Apr-2019_12-03-52
1.4173080533133096% complete.
Transforming image stack Training/Absent/7_Negative_Sample3_Light_16-Apr-2019_18-19-40
1.4266941993617421% complete.
Transforming image stack Training/Absent/XS_Negative_Sample3_Light_17-Apr-2019_12-10-38
1.4360803454101745% complete.
Transforming image stack Training/Absent/XS_Negative_Dark_14-Mar-2019_18-04-46
1.445466491458607% complete.
Transforming image stack Training/Absent/7_Negative_Dark_13-Mar-2019_17-27-28
1.4548526375070396% complete.
Transforming image stack Training/Absent/SE_Negative_Random_03-Jun-2019_16-35-57
1.464238783555472% complete.
Transforming image stack Training/Absent/7_Negative_Random_30-May-2019_16-4

2.0931105688004505% complete.
Transforming image stack Training/Absent/6_Negative_Dark_13-Mar-2019_13-50-05
2.102496714848883% complete.
Transforming image stack Training/Absent/7_Negative_Dark_13-Mar-2019_15-39-59
2.1118828608973157% complete.
Transforming image stack Training/Absent/7_Negative_Sample3_Light_16-Apr-2019_17-22-38
2.1212690069457483% complete.
Transforming image stack Training/Absent/7_Negative_Random_30-May-2019_16-23-35
2.1306551529941804% complete.
Transforming image stack Training/Absent/7_Negative_Dark_13-Mar-2019_16-27-46
2.140041299042613% complete.
Transforming image stack Training/Absent/7_Negative_Random_30-May-2019_16-23-01
2.1494274450910456% complete.
Transforming image stack Training/Absent/XS_Negative_Random_04-Jun-2019_10-24-40
2.158813591139478% complete.
Transforming image stack Training/Absent/6S_Negative_Dark_14-Mar-2019_11-53-43
2.1681997371879107% complete.
Transforming image stack Training/Absent/XS_Negative_Dark_14-Mar-2019_17-33-01
2.17758588323

2.7876853763844562% complete.
Transforming image stack Training/Absent/SE_Negative_Dark_15-Mar-2019_16-13-03
2.797071522432889% complete.
Transforming image stack Training/Absent/XS_Negative_Random_04-Jun-2019_10-20-33
2.806457668481322% complete.
Transforming image stack Training/Absent/6_Negative_Dark_13-Mar-2019_11-54-04
2.8158438145297544% complete.
Transforming image stack Training/Absent/XS_Negative_Dark_14-Mar-2019_17-38-17
2.8252299605781865% complete.
Transforming image stack Training/Absent/6S_Negative_Sample3_Light_16-Apr-2019_15-39-27
2.834616106626619% complete.
Transforming image stack Training/Absent/XS_Negative_Random_30-May-2019_16-00-13
2.8440022526750517% complete.
Transforming image stack Training/Absent/SE_Negative_Dark_15-Mar-2019_16-30-45
2.8533883987234843% complete.
Transforming image stack Training/Absent/6S_Negative_Dark_14-Mar-2019_09-35-37
2.862774544771917% complete.
Transforming image stack Training/Absent/6S_Negative_Random_03-Jun-2019_15-00-04
2.8721606

...

97.05275014079218% complete.
Transforming image stack Training/Present/7_Sample3_Light_27-Mar-2019_14-07-40
97.06213628684063% complete.
Transforming image stack Training/Present/7_Sample4_Dark_12-Mar-2019_12-45-38
97.07152243288905% complete.
Transforming image stack Training/Present/XS_Sample3_Light_28-Mar-2019_13-39-04
97.08090857893748% complete.
Transforming image stack Training/Present/6S_Sample1_Dark_08-Mar-2019_11-45-06
97.09029472498592% complete.
Transforming image stack Training/Present/6_Sample1_Dark_12-Mar-2019_18-34-28
97.09968087103435% complete.
Transforming image stack Training/Present/6S_Sample3_Light_27-Mar-2019_13-10-26
97.10906701708278% complete.
Transforming image stack Training/Present/6_Sample2_Dark_08-Mar-2019_16-54-40
97.1184531631312% complete.
Transforming image stack Training/Present/6S_Sample2_Dark_08-Mar-2019_10-59-41
97.12783930917965% complete.
Transforming image stack Training/Present/XS_Sample3_Light_28-Mar-2019_14-21-44
97.13722545522808% complete.


97.76609724047306% complete.
Transforming image stack Training/Present/6_Sample5_Light_26-Mar-2019_17-33-17
97.77548338652149% complete.
Transforming image stack Training/Present/SE_Sample2_Dark_11-Mar-2019_11-33-11
97.78486953256993% complete.
Transforming image stack Training/Present/6_Sample5_Light_26-Mar-2019_18-01-14
97.79425567861836% complete.
Transforming image stack Training/Present/6_Sample5_Light_26-Mar-2019_17-12-40
97.80364182466678% complete.
Transforming image stack Training/Present/SE_Sample5_Light_01-Apr-2019_15-56-02
97.81302797071523% complete.
Transforming image stack Training/Present/6_Sample4_Dark_08-Mar-2019_15-42-41
97.82241411676365% complete.
Transforming image stack Training/Present/SE_Sample3_Light_28-Mar-2019_15-25-34
97.83180026281208% complete.
Transforming image stack Training/Present/SE_Sample3_Dark_11-Mar-2019_12-19-24
97.84118640886052% complete.
Transforming image stack Training/Present/6S_Sample5_Light_27-Mar-2019_11-21-32
97.85057255490895% complet

98.47944434015393% complete.
Transforming image stack Training/Present/XS_Sample5_Light_27-Mar-2019_17-37-47
98.48883048620236% complete.
Transforming image stack Training/Present/6S_Sample1_Dark_08-Mar-2019_12-02-09
98.4982166322508% complete.
Transforming image stack Training/Present/6_Sample3_Light_25-Mar-2019_14-42-20
98.50760277829923% complete.
Transforming image stack Training/Present/6S_Sample3_Light_27-Mar-2019_13-21-02
98.51698892434766% complete.
Transforming image stack Training/Present/SE_Sample2_Dark_11-Mar-2019_11-03-17
98.5263750703961% complete.
Transforming image stack Training/Present/6S_Sample3_Light_27-Mar-2019_13-19-24
98.53576121644453% complete.
Transforming image stack Training/Present/6_Sample3_Light_25-Mar-2019_14-31-05
98.54514736249295% complete.
Transforming image stack Training/Present/XS_Sample1_Dark_12-Mar-2019_15-39-58
98.5545335085414% complete.
Transforming image stack Training/Present/7_Sample3_Light_27-Mar-2019_14-39-16
98.56391965458982% complete.

99.1927914398348% complete.
Transforming image stack Training/Present/SE_Sample4_Dark_11-Mar-2019_12-29-25
99.20217758588323% complete.
Transforming image stack Training/Present/6S_Sample3_Light_27-Mar-2019_12-58-20
99.21156373193168% complete.
Transforming image stack Training/Present/6_Sample3_Light_25-Mar-2019_16-20-26
99.2209498779801% complete.
Transforming image stack Training/Present/6S_Sample3_Light_27-Mar-2019_12-44-08
99.23033602402853% complete.
Transforming image stack Training/Present/SE_Sample5_Dark_11-Mar-2019_14-40-25
99.23972217007697% complete.
Transforming image stack Training/Present/6S_Sample4_Dark_08-Mar-2019_13-10-25
99.2491083161254% complete.
Transforming image stack Training/Present/7_Sample3_Light_27-Mar-2019_14-10-13
99.25849446217383% complete.
Transforming image stack Training/Present/7_Sample3_Light_27-Mar-2019_15-06-29
99.26788060822226% complete.
Transforming image stack Training/Present/6_Sample3_Dark_08-Mar-2019_16-09-10
99.2772667542707% complete.
Tr

99.90613853951568% complete.
Transforming image stack Training/Present/6S_Sample5_Light_27-Mar-2019_11-18-52
99.91552468556411% complete.
Transforming image stack Training/Present/7_Sample3_Light_27-Mar-2019_14-49-11
99.92491083161254% complete.
Transforming image stack Training/Present/6_Sample4_Dark_08-Mar-2019_15-26-58
99.93429697766098% complete.
Transforming image stack Training/Present/7_Sample3_Light_27-Mar-2019_15-01-59
99.9436831237094% complete.
Transforming image stack Training/Present/XS_Sample5_Light_27-Mar-2019_17-13-18
99.95306926975783% complete.
Transforming image stack Training/Present/6S_Sample2_Dark_08-Mar-2019_11-23-41
99.96245541580628% complete.
Transforming image stack Training/Present/SE_Sample3_Light_28-Mar-2019_16-02-10
99.9718415618547% complete.
Transforming image stack Training/Present/SE_Sample2_Dark_11-Mar-2019_10-38-49
99.98122770790313% complete.
Transforming image stack Training/Present/XS_Sample3_Light_28-Mar-2019_14-37-24
99.99061385395157% complete

Clean up the directory, removing everything but the pickle files.

In [10]:
# Assemble all the pickle files and remove the rest.
classDirs = [direc for direc in os.listdir(topDirName) if direc[0] is not '.']
for classDir in classDirs:
    newDir = os.path.join(topDirName,classDir)
    fileDirs = [direc for direc in os.listdir(newDir) if direc[0] is not '.' and not direc.lower().endswith('.pickle')]
    for fileDir in fileDirs:
        pickleFileDir = os.path.join(newDir,fileDir)
        os.system('rm -rf ' + pickleFileDir)

# Split the directory into train/test/validation sets

In [11]:
import random

In [12]:
finalDirName = 'Split_Data'

In [13]:
def split_folders(inputDir, outputDir, trainValTestRatio, seed):
    # Assume inputDir is organized as 'inputDir/class/topFolder/photos'
    classes = os.listdir(inputDir)
    # Exclude folders where '.' is the first character.
    classes = [directory for directory in classes if directory[0] != '.']
    numClasses = len(classes)
    # Create output directories
    os.system('mkdir ' + outputDir)
    os.system('mkdir ' + outputDir + '/train')
    os.system('mkdir ' + outputDir + '/valid')
    os.system('mkdir ' + outputDir + '/test')
    # Pull files and put into the appropriate folders.
    for directory in classes:
        files = os.listdir(os.path.join(inputDir, directory))
        files.sort()
        random.seed(seed)
        random.shuffle(files)
        numFiles = len(files)
        # Get the indices.
        ndxAmts = np.intc((np.round(trainValTestRatio / np.sum(trainValTestRatio) * numFiles)))
        numNdxes = np.sum(ndxAmts)
        if numNdxes != numFiles:
            ndxAmts[0] += numFiles-numNdxes
        trainFiles = files[0:ndxAmts[0]]
        valFiles = files[ndxAmts[0]:ndxAmts[0]+ndxAmts[1]]
        testFiles = files[ndxAmts[0]+ndxAmts[1]:]
        # Put the training directories in the appropriate folder
        # First make the class directories.
        os.system('mkdir ' + outputDir + '/train/' + directory)
        os.system('mkdir ' + outputDir + '/valid/' + directory)
        os.system('mkdir ' + outputDir + '/test/' + directory)
        for file in trainFiles:
            os.system('cp -r ' + inputDir + '/' + directory + '/' + file + ' ' + outputDir + '/train/' + directory + '/' + file)
        for file in valFiles:
            os.system('cp -r ' + inputDir + '/' + directory + '/' + file + ' ' + outputDir + '/valid/' + directory + '/' + file)
        for file in testFiles:
            os.system('cp -r ' + inputDir + '/' + directory + '/' + file + ' ' + outputDir + '/test/' + directory + '/' + file)

In [14]:
split_folders(topDirName, finalDirName,[0.8,0.1,0.1],7778)